## Read .ghg metadata

In [14]:
import os
import time
import numpy as np
import pandas as pd
import datetime as dt

T1 = time.time()


def find_all_GHG(site):
    raw_path = f'C:/highfreq/{site}/raw/'
    # raw_path = f'Y:/{site}/raw/'
    all_files = []
    all_paths = []

    for (root, dir, files) in os.walk(raw_path):
        if root != raw_path:
            for file in files:
                name, tag = file.split('.')
                # .ghg files are located at the end of each directory tree
                # Avoids reading any that might be misplaced elsewhere
                if tag == 'ghg' and len(dir)==0:
                    # Use the filename as the index
                    all_files.append(name)
                    # Save the relative path of the file
                    all_paths.append(root.replace(raw_path,''))

    df = pd.DataFrame(data={
            'filename':all_files,
            'path':all_paths
            })
    
    AllSiteFiles['datetime'] = pd.to_datetime(df['filename'].str.split('_').str[0])
    df = df.set_index('datetime')
    return (df)

def flag_Maintenance(df):   
    df['Flag'] = 'O'
    df.loc[((df.index.minute!=30)&(df.index.minute!=0)),'Flag'] = 'M'
    flagged = df.loc[df['Flag']=='M']
    flagged.index.date
    flag_dates=pd.to_datetime(flagged.groupby(flagged.index.date).first().index)#.date
    # flag_dates
    # print(flag_dates)
    # df = df.loc[df['Flag']!='M'].resample('30T').first()
    df = df.resample('30T').first()
    return(flag_dates)

df = find_all_GHG('BB2')

df = df.sort_index()

df.to_csv('All_Files_test.csv')

flag_dates = flag_Maintenance(df)

print(time.time()-T1)

print(flag_dates.shape)

# df_flag#.groupby('Flag').count()
df

0.08189868927001953
(10,)


,filename,path,Flag
datetime,,,
2022-05-19 11:35:30,2022-05-19T113530_AIU-1696,2022\05,M
2022-05-19 12:00:00,2022-05-19T120000_AIU-1696,2022\05,O
2022-05-19 12:30:00,2022-05-19T123000_AIU-1696,2022\05,O
2022-05-19 13:00:00,2022-05-19T130000_AIU-1696,2022\05,O
2022-05-19 13:30:00,2022-05-19T133000_AIU-1696,2022\05,O
...,...,...,...
2022-11-28 11:00:00,2022-11-28T110000_AIU-1696,2022\11,O
2022-11-28 11:30:00,2022-11-28T113000_AIU-1696,2022\11,O
2022-11-28 12:00:00,2022-11-28T120000_AIU-1696,2022\11,O


In [1]:
import os
import time
import pandas as pd
import configparser
from Parser import read_GHG
from ipywidgets import FloatProgress
from IPython.display import display

# Site = 'BB2'

# Values we expect to change
dynamicMetaData_Headers = ['canopy_height']

# Values we want to fix
staticMetaData_FloatHeaders = ['altitude', 'latitude', 'longitude']
staticMetaData_TextHeaders = ['site_name']

T1 = time.time()
print('Processing ghg files ')
f = FloatProgress(min=0, max=1) 
display(f) 
# Walk through the directory to find all "raw" folders

N_recs = 4#8*31
RG = read_GHG('Variable_defs.ini')
# def find_new_GHG(Site,make_new=False):

#     i = 0
#     Dpath = 'C:\\highfreq\\'
#     raw_dir = Dpath+Site+'\\raw\\'
#     meta_dir = Dpath+Site+'\\metadata\\'
#     meta_file = meta_dir+'GHGMetaData.csv'
#     channel_file = meta_dir+'EP_Channels.csv'

#     if os.path.isfile(meta_file) and make_new is False:
#         Records = pd.read_csv(meta_file,parse_dates=['TimeStamp'])
#         Channels = pd.read_csv(channel_file)#,parse_dates=['TimeStamp'])
#         FileNames = Records['filename'].tolist()
#     else:
#         Records = pd.DataFrame()
#         Channels = pd.DataFrame()
#         FileNames = []
#         if not os.path.exists(meta_dir):
#             os.mkdir(meta_dir)
    

#     for (root, dirs, files) in sorted(os.walk(raw_dir)):
#         # if root.__contains__('raw') :
#         for file in files:
#             name, tag = file.split('.')
#             # .ghg files are located at the end of each directory tree
#             # Avoids reading any that might be misplaced elsewhere
#             if tag == 'ghg' and len(dirs)==0 and file not in FileNames:  
                
#                 # if i <= 48*5:
#                 if i <= 2:
#                     GHG_out = read_GHG(root,name,ini)
#                     Records = pd.concat(
#                         [Records,GHG_out.Summary],
#                         axis=0,
#                         ignore_index=True
#                     )
#                     Channels = pd.concat(
#                         [Channels,GHG_out.Channels],
#                         axis=0,
#                         ignore_index=True
#                     )
#                     i += 1
#             f.value = i/len(files)
#     return(Records,Channels,meta_dir,i)

# Records,Channels,meta_dir,i = find_new_GHG('BB',make_new=True)

# Records.to_csv(meta_dir+'GHGMetaData.csv',index=False)
# Channels.to_csv(meta_dir+'EP_Channels.csv',index=False)

# elapsed = time.time()-T1
# print('Processing time ', elapsed) 
# print('To inspect ', i, ' .ghg files')
# print('Seconds/file ', elapsed/(i+1)) 

# staticMetaData = Records[staticMetaData_FloatHeaders].astype(float).mean()
# staticMetaData = pd.concat([Records[staticMetaData_TextHeaders].astype(str).mode().T[0],staticMetaData])
# print(staticMetaData)

# dynamicMetaData = Records.set_index('TimeStamp')[dynamicMetaData_Headers]
# dynamicMetaData['date']=dynamicMetaData.index.date
# dynamicMetaData['time']=dynamicMetaData.index.time
# dynamicMetaData.to_csv(meta_dir+'dynamicMetaData.csv',index=False)
# dynamicMetaData

# # Processing time  65.87000775337219
# # To inspect  241  .ghg files

# Records['altitude'].astype(float).describe()


Processing ghg files 


FloatProgress(value=0.0, max=1.0)

Site
Station
Timing
Instruments
Site
Station
Timing
Instruments
Site
Station
Timing
Instruments
Processing time  1.3902320861816406
To inspect  3  .ghg files
Seconds/file  0.34755802154541016
Attribute
site_name    BurnsBog2018
altitude        -7.166667
latitude        49.129368
longitude     -122.984876
dtype: object


count    3.000000
mean    -7.166667
std      0.404145
min     -7.400000
25%     -7.400000
50%     -7.400000
75%     -7.050000
max     -6.700000
Name: altitude, dtype: float64

In [15]:
# Channels = Channels.set_index('filename')
Channels.loc[(Channels.diff()).sum(axis=1)!=0].index


# Channels[(Channels.diff()!=0).sum(axis=0)]
Channels#['TimeStamp'].diff()

C:\Users\User\AppData\Local\Temp\ipykernel_2096\2594540961.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  Channels.loc[(Channels.diff()).sum(axis=1)!=0].index


,CO2 dry(umol/mol),H2O dry(mmol/mol),CH4 (mmol/m^3),N2O (mmol/m^3),Temperature In (C),Temperature Out (C),Total Pressure (kPa),CH4 Temperature,CH4 Pressure,Cell Temperature (C),Diagnostic Value,CH4 Diagnostic Value,Anemometer Diagnostics,Diagnostic Value 2,TimeStamp
filename,,,,,,,,,,,,,,,
2022-06-17T120914_LI-7200.ghg,26,28,50,0,31,32,15,51,52,30,4,54,0,5,2022-06-17 12:09:14
2022-06-17T123000_LI-7200.ghg,26,28,50,0,31,32,15,51,52,30,4,54,0,5,2022-06-17 12:30:00
2022-06-17T130000_LI-7200.ghg,26,28,50,0,31,32,15,51,52,30,4,54,0,5,2022-06-17 13:00:00


In [72]:
import os
# import struct
import numpy as np
import pandas as pd
# print(os.cwd)

os.getcwd()

Dir = 'W:\\2022\\BB\\Flux\\'

Trace = 'co2_mean'
Time = 'clean_tv'

data_file = Dir + Trace
time_file = Dir + Time

# with open(filename, mode='rb') as file:
    # if Trace == 'clean_tv':
Time = np.fromfile(open(time_file, mode='rb'), 'float64')
    # else:
Var = np.fromfile(open(data_file, mode='rb'), 'float32')
# # print(Var.shape,Time.shape)

# df = pd.DataFrame(data = {
#     'Timestamp':Time,
#     'Var':Var
# })

print(Var.shape)
print(2*24*365)

# print(Time.shape)
# Time = Time.reshape(-1,2)
# Time
# pd.to_datetime(Time,unit='s')

    
#     Data = np.array(struct.unpack("i" * ((len(fileContent) -24) // 4), fileContent[20:-4]))

print(Time)
# 738522.02083333
# Matlab Epoch Base ('0-1-0')
base_time = 719529#.000001
pd.to_datetime(Time-base_time,unit='D').round('s')

(17520,)
17520
[738522.02083333 738522.04166667 738522.0625     ... 738886.95833333
 738886.97916667 738887.        ]


DatetimeIndex(['2022-01-01 00:30:00', '2022-01-01 01:00:00',
               '2022-01-01 01:30:00', '2022-01-01 02:00:00',
               '2022-01-01 02:30:00', '2022-01-01 03:00:00',
               '2022-01-01 03:30:00', '2022-01-01 04:00:00',
               '2022-01-01 04:30:00', '2022-01-01 05:00:00',
               ...
               '2022-12-31 19:30:00', '2022-12-31 20:00:00',
               '2022-12-31 20:30:00', '2022-12-31 21:00:00',
               '2022-12-31 21:30:00', '2022-12-31 22:00:00',
               '2022-12-31 22:30:00', '2022-12-31 23:00:00',
               '2022-12-31 23:30:00', '2023-01-01 00:00:00'],
              dtype='datetime64[ns]', length=17520, freq=None)

In [91]:
import re
import configparser

co2app = open('Example_Files\\BB2\\2022-09-01T020000_AIU-1696\\system_config\\co2app.conf').read()

co2app=co2app.replace(')(','),(')[1:-1].replace('))',')\n)')

# co2app=co2app.replace('\n',']\n').replace(')]',')').replace(' ]',']')

# for i in range(4):
#     co2app = co2app.replace('))',')\n')
# co2app = co2app.split('\n')
# for i,c in enumerate(co2app):
#     if ']' in c:
#         co2app[i]='['+co2app[i][1:]

# co2app ='\n'.join(c for c in co2app)
print(co2app)


LI7200(Reg(Reg7 48),(Reg25 32768)
),(Outputs (BW 10),(Delay 0),(InputBW 0),(SDM (Address 0)
),(Dac1 (Source None),(Zero 0),(Full 5),(Set 0)
),(Dac2 (Source None),(Zero 0),(Full 5),(Set 0)
),(Dac3 (Source None),(Zero 0),(Full 5),(Set 0)
),(Dac4 (Source None),(Zero 0),(Full 5),(Set 0)
),(Dac5 (Source None),(Zero 0),(Full 5),(Set 0)
),(Dac6 (Source None),(Zero 0),(Full 5),(Set 0)
),(RS232 (Baud 38400),(Freq 0),(EOL 0D),(Labels TRUE),(DiagRec FALSE),(IM TRUE),(Ndx TRUE),(Time TRUE),(Date TRUE),(CO2Raw FALSE),(H2ORaw FALSE),(DiagVal TRUE),(DiagVal2 TRUE),(CO2D TRUE),(CO2MG TRUE),(H2OD TRUE),(H2OG TRUE),(Temp TRUE),(Pres TRUE),(Aux TRUE),(Aux2 TRUE),(Aux3 TRUE),(Aux4 TRUE),(Cooler TRUE),(ChopperCooler TRUE),(SFVin TRUE),(CO2MF TRUE),(CO2MFd FALSE),(H2OMF TRUE),(H2OMFd FALSE),(DewPt TRUE),(APres FALSE),(DPres FALSE),(AvgTemp FALSE),(TempIn FALSE),(TempOut FALSE),(AvgSS TRUE),(CO2SS TRUE),(H2OSS TRUE),(DeltaSS TRUE),(H2OAW FALSE),(H2OAWO FALSE),(CO2AW FALSE),(CO2AWO TRUE),(MeasFlowRate FALSE),

In [214]:
# def maxDepth(s):
 
#     count = 0
#     st = []
 
#     for i in range(len(s)):
#         if (s[i] == '('):
#             st.append(i) # pushing the bracket in the stack
#         elif (s[i] == ')'):
#             if (count < len(st)):
#                 count = len(st)
#             # keeping track of the parenthesis and storing
#             # it before removing it when it gets balanced
#             st.pop()
         
#     return count
 
# Driver program
 
co2app = open('Example_Files\\BB\\2022-08-01T010000_LI-7200\\system_config\\co2app.conf').read()

# s = "( ((X)) (((Y))) )"
md = (maxDepth(co2app))

# co2app = co2app.replace(')))))',')))))\n')
# print(maxDepth(co2app))
# print(co2app)
# re.search()
# for i in range(2):
#     co2app = re.search(r'\((.*)\)',co2app).group(1)
#     print(len(co2app))
# co2app = co2app.replace('(','{').replace(')','}')
all = re.findall(r'\((.+?)\)',co2app)

formatted =''
for i,v in enumerate(all):
    if '(' in v:
        # if i != 0:
        #     formatted += '\n'
        tags = v.split('(')[:-1]
        for t in tags:
            # formatted += '\n'
            formatted += '\n['+t.replace(' ','')+']\n'
        formatted += v.split('(')[-1].replace(' ','=',1)
        # pass
    else:
        formatted += '\n'+v.replace(' ','=',1)

# vals = {}
formatted = formatted.replace('\n\n','\n')#.replace('\n[','\n\n[')


config = configparser.ConfigParser()
config.read_string(formatted)
print(config)
# with open('Example_Files\\BB\\2022-08-01T010000_LI-7200\\system_config\\co2app_formatted.conf', 'w') as configfile:
#     configfile.write(formatted)

DuplicateOptionError: While reading from '<string>' [line 231]: option 'temp' in section 'Logging' already exists

In [234]:
def make_Dict(string):
    all = re.findall(r'\((.+?)\)',string)
    formatted =''
    for i,v in enumerate(all):
        if '(' in v:
            # if i != 0:
            #     formatted += '\n'
            tags = v.split('(')[:-1]
            for t in tags:
                # formatted += '\n'
                formatted += '\n['+t.replace(' ','')+']\n'
            formatted += v.split('(')[-1].replace(' ','=',1)
            # pass
        else:
            formatted += '\n'+v.replace(' ','=',1)
    print(formatted)
co2app = open('Example_Files\\BB\\2022-08-01T010000_LI-7200\\system_config\\co2app.conf').read()
# Coefs = co2app.split('Coef')[-1]
Calibration = co2app.split('Calibrate')[-1]
Coefs = Calibration.split('Coef')[-1].split('(Clock')[0][:-1]
Calibration = Calibration.split('Coef')[0][1:-2]
make_Dict(Coefs)


[Current]
SerialNo=72H-0816
[Band]
A=1.15
[CO2]
A=1.05790E+2
B=1.85532E+4
C=8.28945E+6
D=-1.15799E+9
E=1.00378E+11
XS=0.0028
Z=3.84E-05
SD1=3.230E-2
SD2=-6.330E-2
SD3=2.085E+0
[H2O]
A=5.59192E+3
B=5.95452E+6
C=-5.74709E+8
XS=-0.0012
Z=-1.96E-04
SD1=1.910E-2
SD2=1.323E+0
SD3=2.385E+0
[Pressure]
A0=57.704
A1=15.424
[DPressure]
A0=0.000
A1=1.000
SerialNo=72H-0816
[MaxRef]
B=0.050
C=0.051


In [241]:
config = configparser.ConfigParser()
config.read_file(open('Variable_defs.ini'))
for v in config['METADATA']:
    print(v)
    print(config['METADATA'][v])

sections
['Site','Station','Timing','Instruments']
